# Egobox

## Installation

You can install with the `pip` command as follows: 

```bash
pip install egobox
```

## Import

In [1]:
import numpy as np
import egobox as egx

# To display optimization information (none by default)
import logging
logging.basicConfig(level=logging.INFO)

## Continuous test functions 

In [2]:
xspecs_xsinx = egx.to_specs([[0., 25.]])
n_cstr_xsinx = 0

def xsinx(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    y = (x - 3.5) * np.sin((x - 3.5) / (np.pi))
    return y

In [3]:
xspecs_g24 = egx.to_specs([[0., 3.], [0., 4.]])
n_cstr_g24 = 2

# Objective
def G24(point):
    """
    Function g24
    1 global optimum y_opt = -5.5080 at x_opt =(2.3295, 3.1785)
    """
    p = np.atleast_2d(point)
    return - p[:, 0] - p[:, 1]

# Constraints < 0
def G24_c1(point):
    p = np.atleast_2d(point)
    return (- 2.0 * p[:, 0] ** 4.0
            + 8.0 * p[:, 0] ** 3.0 
            - 8.0 * p[:, 0] ** 2.0 
            + p[:, 1] - 2.0)

def G24_c2(point):
    p = np.atleast_2d(point)
    return (-4.0 * p[:, 0] ** 4.0
            + 32.0 * p[:, 0] ** 3.0
            - 88.0 * p[:, 0] ** 2.0
            + 96.0 * p[:, 0]
            + p[:, 1] - 36.0)

# Grouped evaluation
def g24(point):
    p = np.atleast_2d(point)
    return np.array([G24(p), G24_c1(p), G24_c2(p)]).T


## Ego optimizer written in Rust (a.k.a Egor)

In [4]:
egor = egx.Egor(g24, xspecs_g24, 
                     n_doe=10, 
                     n_cstr=n_cstr_g24, 
                     cstr_tol=1e-3,
                     infill_strategy=egx.InfillStrategy.WB2,
                     # expected=egx.ExpectedOptimum(val=-5.50, tol=1e-2),
                     # outdir="./out",
                     # hot_start=True
                    )  # see help(egor) for options

# Restrict regression and correlation models used
#egor = egx.Egor(g24, xlimits_g24, n_cstr=n_cstr_g24, n_doe=10,
#                      regr_spec=egx.RegressionSpec.LINEAR,
#                      corr_spec=egx.CorrelationSpec.MATERN32 | egx.CorrelationSpec.MATERN52)  

In [5]:
res = egor.minimize(n_eval=30)
print(f"Optimization f={res.y_opt} at {res.x_opt}")

INFO:egobox_ego.egor_solver:Compute initial LHS with 10 points
INFO:egobox_ego.egor_solver:INITIAL STATE = EgorState { param: None, prev_param: None, best_param: None, prev_best_param: None, cost: None, prev_cost: None, best_cost: None, prev_best_cost: None, target_cost: -inf, iter: 0, last_best_iter: 0, max_iters: 20, counts: {}, time: Some(0ns), termination_reason: NotTerminated, added: 10, prev_added: 0, no_point_added_retries: 3, lhs_optim: false, clusterings: Some([None, None, None]), data: Some(([[0.08117349164416486, 0.36910338877124843],
 [1.0346721049978354, 3.5481259414749475],
 [0.7268346743387302, 0.574211279983301],
 [2.5831805574492552, 0.9676616787277155],
 [2.9660187679424816, 2.8840883967958493],
 [1.2259374755386956, 1.9754195523457247],
 [1.8596332391405768, 2.6700484263603217],
 [2.266110518355646, 3.7658868942713903],
 [1.7100454434887629, 2.121490775670748],
 [0.5003759864224575, 1.5597654739966529]], shape=[10, 2], strides=[2, 1], layout=Cc (0x5), const ndim=2, [

Optimization f=[-5.497894505529171, -0.013718712731103366, -0.008665669319398717] at [2.3299127192477673, 3.1679817862814037]


## Mixed integer test function

In [6]:
xspecs_mixint_xsinx = [egx.Vspec(egx.Vtype(egx.Vtype.INT), [0, 25])]
n_cstr_mixint_xsinx = 0

def mixint_xsinx(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    if (np.abs(np.linalg.norm(np.floor(x))-np.linalg.norm(x))< 1e-8):
        y = (x - 3.5) * np.sin((x - 3.5) / (np.pi))
    else:
        raise ValueError(f"Bad input: mixint_xsinx accepts integer only, got {x}")
    return y

In [7]:
egor = egx.Egor(mixint_xsinx, xspecs_mixint_xsinx, 
                     n_doe=3, 
                     infill_strategy=egx.InfillStrategy.EI,
                     target=-15.12,
                    )  # see help(egor) for options
res = egor.minimize(n_eval=30)
print(f"Optimization f={res.y_opt} at {res.x_opt}")

INFO:egobox_ego.egor_solver:Compute initial LHS with 3 points
INFO:egobox_ego.egor_solver:INITIAL STATE = EgorState { param: None, prev_param: None, best_param: None, prev_best_param: None, cost: None, prev_cost: None, best_cost: None, prev_best_cost: None, target_cost: -15.12, iter: 0, last_best_iter: 0, max_iters: 27, counts: {}, time: Some(0ns), termination_reason: NotTerminated, added: 3, prev_added: 0, no_point_added_retries: 3, lhs_optim: false, clusterings: Some([None]), data: Some(([[20.889936565704932],
 [11.638314658847898],
 [2.6544943376206387]], shape=[3, 1], strides=[1, 1], layout=CFcf (0xf), const ndim=2, [[-11.443706815463152],
 [3.589376194523597],
 [0.07924194329580243]], shape=[3, 1], strides=[1, 1], layout=CFcf (0xf), const ndim=2)), sampling: Some(Lhs { xlimits: [[0.0, 25.0]], shape=[1, 2], strides=[2, 1], layout=CFcf (0xf), const ndim=2, kind: Maximin, rng: Xoshiro256Plus { s: [16638544911602489356, 117542120744815069, 13290965817535454840, 17930214200403379493] }

Optimization f=[-15.121611536143384] at [19.0]


## Usage

In [8]:
help(egor)

Help on Egor in module builtins object:

class Egor(object)
 |  Egor(fun, n_cstr=0, cstr_tol=1e-06, n_start=20, n_doe=0, regression_spec=7, correlation_spec=15, infill_strategy=1, q_parallel=1, par_infill_strategy=1, infill_optimizer=1, n_clusters=1)
 |  
 |  Optimizer constructor
 |  
 |  fun: array[n, nx]) -> array[n, ny]
 |       the function to be minimized
 |       fun(x) = [obj(x), cstr_1(x), ... cstr_k(x)] where
 |          obj is the objective function [n, nx] -> [n, 1]
 |          cstr_i is the ith constraint function [n, nx] -> [n, 1]
 |          an k the number of constraints (n_cstr)
 |          hence ny = 1 (obj) + k (cstrs)
 |       cstr functions are expected be negative (<=0) at the optimum.
 |  
 |   n_cstr (int):
 |       the number of constraint functions.
 |  
 |   cstr_tol (float):
 |       tolerance on constraints violation (cstr < tol).
 |  
 |   xspecs (list(Vspec)) where Vspec(vtype=FLOAT|INT, vlimits=[lower bound, upper bound]):
 |       Bounds of the nx compo